# Explorer eigenvals

In [1]:
%pylab inline

import h5py
import numpy as np
from isingchat.io import read_ising_config
from isingchat import ising
from matplotlib import pyplot, rcParams
from matplotlib.axes import Axes, GridSpec
from matplotlib.figure import Figure
from ruamel.yaml import YAML
from scipy import interpolate as interp
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import pandas as pd
import json

import os
import re


Populating the interactive namespace from numpy and matplotlib


In [7]:
from isingchat.ising import _csr_finite_log_transfer_matrix_parts_fast,csr_matrix,eigvals,_centrosym_log_transfer_matrix_parts_fast
from scipy import linalg
temp = 0.5
mag_field = 0
interactions = np.array([1.0, 0.5,0.33]) # p=1
num_neighbors = 3

nnz_elems, nnz_rows, nnz_cols = _centrosym_log_transfer_matrix_parts_fast(
        temp, interactions, num_neighbors
)

# Normalize nonzero matrix elements.
max_w_log_elem = np.max(nnz_elems)
nnz_elems -= max_w_log_elem
norm_nnz_elems = np.exp(nnz_elems)
# Construct the sparse matrix.
num_rows = 2 ** (num_neighbors-1)
w_shape = (num_rows, num_rows)
w_matrix = csr_matrix(
    (norm_nnz_elems, (nnz_rows, nnz_cols)), shape=w_shape
)
# Evaluate all eigenvalue only.
eigenval, eigenvect = linalg.eig(w_matrix.todense())
print("eigenvals: {}\neigenvect: {}".format(eigenval,eigenvect))


eigenvals: [ 1.00000684+0.j          0.02093335+0.j         -0.00802371+0.01373809j
 -0.00802371-0.01373809j]
eigenvect: [[ 9.99999780e-01+0.j          2.59888345e-01+0.j
  -2.51381001e-01-0.00342601j -2.51381001e-01+0.00342601j]
 [ 2.55905296e-05+0.j         -9.52506129e-01+0.j
   9.48751719e-01+0.j          9.48751719e-01-0.j        ]
 [ 1.21885524e-05+0.j         -1.19397871e-01+0.j
  -3.77112346e-02+0.12797431j -3.77112346e-02-0.12797431j]
 [ 6.62220971e-04+0.j         -1.04567060e-01+0.j
  -6.93957208e-02-0.11853567j -6.93957208e-02+0.11853567j]]


In [48]:
order_eig = np.array([eigenvect[1],eigenvect[6],
                      eigenvect[0],eigenvect[7],                      
                     eigenvect[2],eigenvect[4],
                    eigenvect[3],eigenvect[5]])

# order_eig = np.array([eigenvect[1],eigenvect[2],
#                       eigenvect[0],eigenvect[3]])
print(order_eig)

[[ 0.        +0.j          0.        -0.j          0.        +0.j
  -0.70241032+0.j         -0.70241032-0.j          0.70241032+0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.70241032+0.j          0.70241032-0.j          0.70241032+0.j
   0.        +0.j          0.        -0.j          0.        +0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        -0.j          0.        +0.j
   0.        +0.j          0.        -0.j          0.        +0.j
   1.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        -0.j          0.        +0.j
   0.        +0.j          0.        -0.j          0.        +0.j
   0.        +0.j          1.        +0.j        ]
 [ 0.        +0.j          0.        -0.j          0.        +0.j
   0.25164949-0.43586971j  0.25164949+0.43586971j  0.50329899+0.j
   0.        +0.j          0.        +0.j        ]
 [ 0.        +0.j          0.        -0.j          0.        +0.j
   0.25164949+0.435

In [49]:
sigma_z = np.array([
   [1,0],
    [0,-1],
])
identity = np.array([
    [1,0],
    [0,1]
])
sig_gen = np.kron(sigma_z,identity)
for i in range(num_neighbors-2):
    sig_gen = np.kron(sig_gen,identity)
print("sigma_gen: {}".format(sig_gen))

# unitary_mat = np.matrix.transpose(eigenvect)
unitary_mat = np.matrix.transpose(order_eig)
inverse_unitary_mat = linalg.inv(unitary_mat)
np.dot(np.dot(unitary_mat,sig_gen),inverse_unitary_mat)

sigma_gen: [[ 1  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0 -1]]


array([[-0.33333333-4.40024708e-16j,  0.66666667+3.92642560e-16j,
         0.66666667+4.73821483e-17j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.66666667-3.92642560e-16j, -0.33333333+3.56757982e-16j,
         0.66666667+6.36401542e-17j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.66666667-3.16106240e-16j,  0.66666667+2.60595089e-16j,
        -0.33333333+3.89916055e-17j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, -0.33333333-4.48153711e-16j,
         0.66666667+4.12269133e-16j, -0.66666667-8.12900297e-18j,
       

In [29]:
A=np.array([
    [1,2],
    [3,4]
]) 
B=np.array([
    [1,0],
    [1,1]
])
np.dot(A,B)

array([[3, 2],
       [7, 4]])

In [20]:
import numpy